In [97]:
import xml.etree.cElementTree as ET
import os
import re
import pandas as pd
import json
import random
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import subprocess

random.seed(0)

apertium_mt_path = '.'

In [98]:
def txt_file_read(filename):
    lines=[]
    with open(filename, "r") as f:
        for line in f:
            line=line.strip()
            lines.append(line)
    return lines

In [99]:
def rule_machine_translation(sux_sentence,eng_tranlation_reference):
    sux_sentence = sux_sentence.replace(':','-').replace('(','\(').replace(')','\)').replace("'","\\'").replace('|','\|')
    apertium_translation_command = f'''echo {sux_sentence} | apertium -d {apertium_mt_path} sux-eng'''
    output = subprocess.check_output(apertium_translation_command, shell=True)
    output = output.decode('ascii').strip().replace('#','').replace('-',' ').replace('*',' ').split()

    return " ".join(output)

In [100]:
def nn_machine_translation(sux_sentence,eng_tranlation_reference):
    return eng_tranlation_reference

In [101]:
dir_path = 'data/consolidated/dev'
file_name = os.listdir(dir_path)
file_name

['P320163.conll',
 'P125272.conll',
 'P416458.conll',
 'P107372.conll',
 'P101172.conll',
 'P203075.conll',
 'P108689.conll',
 'P206146.conll',
 'P118757.conll',
 'P209503.conll',
 'P432333.conll',
 'P273892.conll',
 'P432157.conll',
 'P107130.conll',
 'P107369.conll',
 'P432372.conll',
 'P134336.conll',
 'P361739.conll',
 'P432116.conll',
 'P332170.conll',
 'P407068.conll',
 'P470071.conll',
 'P331046.conll',
 'P467836.conll',
 'P340847.conll',
 'P316855.conll',
 'P315822.conll',
 'P424392.conll',
 'P411977.conll',
 'P432188.conll',
 'P430147.conll',
 'P208745.conll',
 'P137383.conll',
 'P116255.conll',
 'P204646.conll',
 'P432290.conll',
 'P113323.conll',
 'P432335.conll',
 'P107128.conll',
 'P432353.conll',
 'P331043.conll',
 'P130219.conll',
 'P212352.conll',
 'P205148.conll',
 'P370955.conll',
 'P108651.conll',
 'P100765.conll',
 'P127326.conll',
 'P432169.conll',
 'P432268.conll',
 'P458725.conll',
 'P111481.conll',
 'P142051.conll',
 'P430412.conll',
 'P465776.conll',
 'P416464.

In [107]:
translation_data = []
count=0
sum_rule_bleu = 0
sum_nn_bleu = 0
chencherry = SmoothingFunction()


for file in file_name:
    count+=1
    print(count)

    file_path = os.path.join(dir_path,file)
    file_data = txt_file_read(file_path)

    # extracting sumerian and english translation from the file
    sux_sentence = ''
    for row in file_data:
        if row.startswith('# tr.en'):
            eng_tranlation_reference = row.split('tr.en:')[1]
        row_line = row.split('\t')
        if row_line[0].isdigit() and 'XPOSTAG' not in row:
            sux_sentence+=row_line[1]+' '

    # basic cleaning of english reference sentence so we do not miss correct words match because of basic errors like (Su-zen same as Suzen and suzen same as suzen)
    eng_tranlation_reference = eng_tranlation_reference.replace('-','').lower()

    # basic cleaning of sumerian sentence before passing to rulebased translation
    sux_sentence = sux_sentence.replace('<','').replace('>','').lower()


    #================================ Machine Translation ================================================ #


    eng_rule_based_translation = rule_machine_translation(sux_sentence,eng_tranlation_reference)
    rule_bleu = sentence_bleu([eng_tranlation_reference.split()], eng_rule_based_translation.split(),smoothing_function=chencherry.method1,weights = (0.75,0.25,0,0))*100
    sum_rule_bleu += rule_bleu
    avg_rule_bleu = sum_rule_bleu/count 

    eng_nn_based_translation = nn_machine_translation(sux_sentence,eng_tranlation_reference)
    nn_bleu = sentence_bleu([eng_tranlation_reference.split()], eng_nn_based_translation.split(), smoothing_function=chencherry.method1, weights = (0.75,0.25,0,0))*100
    sum_nn_bleu += nn_bleu
    avg_nn_bleu = sum_nn_bleu/count 

    translation_data.append([file, sux_sentence, eng_tranlation_reference, eng_rule_based_translation, eng_nn_based_translation, rule_bleu, nn_bleu])

    # if count==33:
    #     break

print(f'''\n average bleu score for rule based is - {avg_rule_bleu}''')

1
2
3


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155


156
157

 average bleu score for rule based is - 6.244483513924998


In [108]:
col_name = ['file', 'sux_sentence', 'eng_tranlation_reference', 'eng_rule_based_translation', 'eng_nn_based_translation', 'rule_bleu', 'nn_bleu']
trainslation_pd = pd.DataFrame(translation_data,columns = col_name)
trainslation_pd.to_csv('translation_results.csv')

In [109]:
trainslation_pd.head()

,file,sux_sentence,eng_tranlation_reference,eng_rule_based_translation,eng_nn_based_translation,rule_bleu,nn_bleu
0,P320163.conll,pisan-dub-ba mu 2(disz) sze-ba giri3-se3-ga ug...,basketoftablets years of rations personnel of...,archivist name one ration attendant ugnim{ pla...,basketoftablets years of rations personnel of...,0.000000,100.0
1,P125272.conll,usz2 ur-sila-luh 1(asz@c) gan2 e2-ur2-bi-du10 ...,dead ursilalu ac field eurbidu a foreman ac f...,he dead ur unit luh 1( asz@ c) gan2 E'urbidu o...,dead ursilalu ac field eurbidu a foreman ac f...,6.754154,100.0
2,P416458.conll,4(disz) ki szu-{d}idim-ta mu-kux(du) iti ezem-...,oxen cows male equids female equids old from ...,one place hand { d} idim unknown name kux(he g...,oxen cows male equids female equids old from ...,2.645627,100.0
3,P107372.conll,pisan-dub-ba kiszib3 didli masz-x-x e2 lu2-gi-...,basketoftablets sealed documents varied from ...,archivist seal he several goat _ house Lugina ...,basketoftablets sealed documents varied from ...,4.413135,100.0
4,P101172.conll,5(disz) sila3 kasz saga 5(disz) sila3 ninda 5(...,sila fine beer sila bread shekels onions shek...,one unit beer he good one unit bread one unit ...,sila fine beer sila bread shekels onions shek...,8.348981,100.0


# Exrta

In [16]:
# result = os.system(f'''echo "e2 lugal-la mu-un-du3" | apertium -d apertium-sux-eng/ sux-eng''')

In [51]:
sux_sentence = "pisan-dub-ba dub gid2-da i3-dub giri3 ku5-da-mu i3-gal2 mu si-mu-ru-um{ki} lu-lu-bu{ki} <a>-ra2 1(u) la2 1(disz)-kam-asz ba-hul"

In [52]:
sux_sentence = sux_sentence.replace('<','').replace('>','')
sux_sentence

'pisan-dub-ba dub gid2-da i3-dub giri3 ku5-da-mu i3-gal2 mu si-mu-ru-um{ki} lu-lu-bu{ki} a-ra2 1(u) la2 1(disz)-kam-asz ba-hul'

In [53]:
sux_sentence = sux_sentence.replace(':','-').replace('(','\(').replace(')','\)').replace("'","\\'").replace('|','\|')
apertium_translation_command = f'''echo {sux_sentence} | apertium -d . sux-eng'''
output = subprocess.check_output(apertium_translation_command, shell=True)
output = output.decode('ascii').strip().replace('#','').replace('-',' ').replace('*',' ').split()
" ".join(output)

'archivist tablet he drag granary foot Kudamu he be in name Simurrum Lulubu times ten he hang one asz he _'

### NLTK BlEU

In [ ]:

reference = [['this','boy']]
candidate = ['this', 'is']
chencherry = SmoothingFunction()
score = sentence_bleu(reference, candidate, smoothing_function=chencherry.method1, weights=(1,0,0,0))
print(score)

0.5


In [ ]:
sentence_bleu([eng_tranlation_reference.split()], eng_nn_based_translation.split(), smoothing_function=chencherry.method1)

### Huggingface evaluate

In [ ]:
import evaluate

/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
predictions = ["hello there"]
references = [
     ["hello bro"],
 ]
bleu = evaluate.load("sacrebleu")
results = bleu.compute(predictions=predictions, references=references, smooth_method= 'floor')
print(results)

{'score': 0.0, 'counts': [2, 1, 0, 0], 'totals': [2, 1, 0, 0], 'precisions': [100.0, 100.0, 0.0, 0.0], 'bp': 1.0, 'sys_len': 2, 'ref_len': 2}


In [ ]:
predictions = ["hello"]
references = [["hello there"]]
sacrebleu = evaluate.load("sacrebleu")
results = sacrebleu.compute(predictions=predictions, 
                             references=references, smooth_method = 'add-k')

In [ ]:
results

{'score': 36.78794411714425,
 'counts': [1, 1, 1, 1],
 'totals': [1, 1, 1, 1],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 0.36787944117144233,
 'sys_len': 1,
 'ref_len': 2}